<a href="https://colab.research.google.com/github/amandugar/auto-mobiles/blob/main/automobiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch-lr-finder
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if(os.path.isdir('/content/final-dataset-automobiles/')):
  print ('exist')
else:
  !cp '/content/drive/MyDrive/final-dataset-automobiles.zip' '/content'


In [ ]:
if(os.path.isdir('/content/final-dataset-automobiles/')):
  print('exist')
else:
  !unzip final-dataset-automobiles.zip

In [ ]:
!rm '/content/final-dataset-automobiles.zip'

In [ ]:
import numpy as np
import torch
from torch_lr_finder import LRFinder
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image, ImageDraw
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
import os
import cv2
import pathlib
import torch.optim as optim

In [ ]:
!rm -rf '/content/final-dataset-automobiles/Air Bag Light and Dashboard'

In [ ]:
path = "final-dataset-automobiles"
for folder in os.listdir(path):
  for img_file in os.listdir(os.path.join(path,folder)):
    img_file = os.path.join(path,folder,img_file)
    try:
      img = Image.open(img_file)
      if img.mode!='RGB':
        os.remove(img_file)
        print(img_file)
    except:
      print(img_file)
      os.remove(img_file)

In [ ]:
transform = transforms.Compose([
                                transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5],[0.5])
                              ])
# transform2 = transforms.Compose([
#                                 transforms.Resize(255),
#                                 transforms.CenterCrop(224),
#                                 transforms.ToTensor(),
#                                 transforms.Normalize([0.5],[0.5]),
#                                 transforms.RandomRotation((0,180))
# ])

In [ ]:
dataset = datasets.ImageFolder('final-dataset-automobiles',transform=transform)
# dataset2 = datasets.ImageFolder('final-dataset-automobiles',transform=transform2)
dataset_len = len(dataset)
dataset_len

5109

In [ ]:
train_len,test_len = dataset_len - 1000,1000
train_set,test_set = torch.utils.data.random_split(dataset,[train_len,test_len],generator=torch.Generator().manual_seed(30))
batch_size = 16
train_set = DataLoader(dataset=train_set,shuffle=True,batch_size=batch_size)
test_set = DataLoader(dataset=test_set,shuffle=True,batch_size=batch_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using Device: ', device)

Using Device:  cpu


In [ ]:
root=pathlib.Path('/content/final-dataset-automobiles')
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
classes

['2nd Row Left Side Seat Armrest',
 '2nd Row Right Side Seat Armrest',
 '2nd Row Seat Belt (L)',
 '2nd Row Seat Belt (R)',
 '2nd Row Seat Belt Buckle Right',
 'AUX - USB Socket',
 'Accelerator Pedal',
 'Accessory Socket (Centre Console)',
 'Accessory Socket (Dash Board)',
 'Accessory Socket (Inside Armrest)',
 'Accessory socket location (luggage compartment)',
 'Air Cleaner',
 'Airbag Label',
 'Audio System',
 'Bill - Coin Holder',
 'Brake - Clutch Fluid Reservoir',
 'Brake Pedal',
 'Centre Outlet (L)',
 'Centre Outlet (R)',
 'Child-Proof Locks (Rear L)',
 'Child-Proof Locks (Rear R)',
 'Coolant Reservoir',
 'Courtesy Lights (Duplicate)',
 'Door Locks (Driver side - O)',
 'Door Locks (Front Passenger side - I)',
 'Door Locks (Front Passenger side - O)',
 'Door Locks (Rear Left)',
 'Door Locks (Rear Right)',
 'Driver Door Switch',
 'Driver Side Defroster Outlet',
 'Driver Side Outlet',
 'Driver Side Seat Belt Buckle',
 'Engine Hood Latch',
 'Engine Oil Filler Cap',
 'Front Dome Light',


In [ ]:
len(classes)

37

In [ ]:
model = models.resnet18(pretrained=True)

In [ ]:
for param in model.parameters(): 
  param.requires_grid = False 
  model.fc = torch.nn.Linear(in_features=512,out_features=len(classes),bias=True) 
  model = model.to(device)

In [ ]:
list(model.parameters())[0].shape

torch.Size([64, 3, 7, 7])

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=1E-06,weight_decay=10e-5)

In [ ]:
lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
lr_finder.range_test(train_set, end_lr=100, num_iter=100)
lr_finder.plot() # to inspect the loss-learning rate graph
lr_finder.reset()

KeyboardInterrupt: ignored

In [ ]:
model.train()
for epoch in range(10):
  total_correct = 0.0
  running_loss = 0.0
  for i,(inputs,labels) in enumerate(train_set):
    inputs,labels = inputs.to(device),labels.to(device)
    output = model(inputs)
    output_idx = torch.argmax(output,dim=1)
    total_correct+=(labels == output_idx).sum().item()
    optimizer.zero_grad()
    loss = criterion(output,labels)
    running_loss+= loss.item()*inputs.size(0)
    loss.backward()
    optimizer.step()
  print(f'Epoch: {epoch} Loss = {running_loss/train_len} Accuracy:{(total_correct/train_len)*100}%')

Epoch: 0 Loss = 0.0319056120889484 Accuracy:98.02871744950109%
Epoch: 1 Loss = 0.03097016066466915 Accuracy:98.07739109272329%
Epoch: 2 Loss = 0.031618422888678104 Accuracy:98.15040155755659%
Epoch: 3 Loss = 0.030655521929493442 Accuracy:98.32075930883427%
Epoch: 4 Loss = 0.030471837309448208 Accuracy:98.24774884400097%
Epoch: 5 Loss = 0.030099852725475426 Accuracy:98.46678023850085%
Epoch: 6 Loss = 0.031041271127823875 Accuracy:98.27208566561208%
Epoch: 7 Loss = 0.03044112562422905 Accuracy:98.22341202238988%
Epoch: 8 Loss = 0.0307496782559081 Accuracy:98.17473837916768%
Epoch: 9 Loss = 0.030572845015097314 Accuracy:98.07739109272329%


In [ ]:
with torch.no_grad():
  model.eval()
  total_loss = 0.0
  total_correct = 0.0
  for inputs,labels in test_set:
    labels = labels.to(device)
    outputs = model(inputs.to(device))
    loss = criterion(outputs,labels)
    total_loss = loss.item() * inputs.size(0)
    output_idx = torch.argmax(outputs,dim=1)
    total_correct+=sum(labels == output_idx)
print(f'Loss = {running_loss/test_len} Accuracy:{(total_correct/test_len)*100}%')

Loss = 0.12562382016703486 Accuracy:95.50000762939453%


In [ ]:
torch.save(model.state_dict(),'drive/MyDrive/auto-170epoch-new.pt')

In [ ]:
import numpy as np
import torch
from torchvision import models,transforms
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
labels = ['2nd Row Left Side Seat Armrest','2nd Row Right Side Seat Armrest','2nd Row Seat Belt (L)',
 '2nd Row Seat Belt (R)',
 '2nd Row Seat Belt Buckle Right',
 'AUX - USB Socket',
 'Accelerator Pedal',
 'Accessory Socket (Centre Console)',
 'Accessory Socket (Dash Board)',
 'Accessory Socket (Inside Armrest)',
 'Accessory socket location (luggage compartment)',
 'Air Cleaner',
 'Airbag Label',
 'Audio System',
 'Bill - Coin Holder',
 'Brake - Clutch Fluid Reservoir',
 'Brake Pedal',
 'Centre Outlet (L)',
 'Centre Outlet (R)',
 'Child-Proof Locks (Rear L)',
 'Child-Proof Locks (Rear R)',
 'Coolant Reservoir',
 'Courtesy Lights (Duplicate)',
 'Door Locks (Driver side - O)',
 'Door Locks (Front Passenger side - I)',
 'Door Locks (Front Passenger side - O)',
 'Door Locks (Rear Left)',
 'Door Locks (Rear Right)',
 'Driver Door Switch',
 'Driver Side Defroster Outlet',
 'Driver Side Outlet',
 'Driver Side Seat Belt Buckle',
 'Engine Hood Latch',
 'Engine Oil Filler Cap',
 'Front Dome Light',
 'Second Row Seat Belt Buckle (Left)',
 'Sound System']
model_path = '/content/drive/MyDrive/auto.pt'
img_file_path = '/content/final-dataset-automobiles/Courtesy Lights (Duplicate)/20210629_123227-resized.jpg'
transform = transforms.Compose([transforms.CenterCrop(224),transforms.ToTensor()])
model = models.resnet18()
print (len(labels))
model.fc = torch.nn.Linear(in_features=512,out_features=len(labels),bias=True)
model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
model.eval()
img = Image.open(img_file_path)
img = transform(img)
prediction = model(img.unsqueeze(0))
print(prediction)
result = torch.argmax(prediction)
print(result)
print(f'This is a {labels[result]}')

37
tensor([[-1.4972, -2.1163, -1.2768, -2.6181, -0.6688, -1.2867, -0.6133, -0.7445,
         -1.3571, -0.6131, -1.5266, -1.8204, -1.2371, -0.0123, -0.8968, -2.1421,
         -2.9173,  1.2454, -2.1740, -0.7510, -2.2874, -1.0214,  6.8698, -0.3277,
         -2.1850, -1.9870, -1.0510, -1.3311, -2.0658, -2.7467, -1.3517, -2.3081,
         -1.5043, -2.7032,  3.0109, -2.0962, -2.8809]],
       grad_fn=<AddmmBackward>)
tensor(22)
This is a Courtesy Lights (Duplicate)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
import torch
from torchvision import models

In [ ]:
!pip install onnx
!pip install onnxruntime

     |████████████████████████████████| 12.2 MB 241 kB/s 
     |████████████████████████████████| 4.5 MB 7.5 MB/s 


In [ ]:
pt_model = models.resnet18()
pt_model.fc = torch.nn.Linear(in_features=512,out_features=37,bias=True)
pt_model.load_state_dict(torch.load('/content/drive/MyDrive/auto-170epoch-new.pt',map_location=torch.device('cpu')))

dummy_input = torch.randn(64,3,224,224,device='cpu')
torch.onnx.export(pt_model,               # model being run
                  dummy_input,                         # model input (or a tuple for multiple inputs)
                  "/content/drive/MyDrive/auto-170new.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=8,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [ ]:
import numpy
import onnxruntime
import onnx

In [ ]:
ort_session = onnxruntime.InferenceSession("/content/drive/MyDrive/auto-170new.onnx")
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [ ]:
img_file_path = '/content/final-dataset-automobiles/2nd Row Seat Belt (R)/20210615_135808-resized.jpg'
transform = transforms.Compose([transforms.CenterCrop(224),transforms.ToTensor()])
img = Image.open(img_file_path)
print(np.asarray(img))
img = transform(img)
img = img.unsqueeze(0)
print(img.shape)
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img)}
ort_outs = ort_session.run(None, ort_inputs)[0]

[[[ 52  58  56]
  [ 60  66  64]
  [ 71  77  75]
  ...
  [118 120 119]
  [115 117 116]
  [119 121 120]]

 [[ 45  51  49]
  [ 50  56  54]
  [ 57  63  61]
  ...
  [117 119 118]
  [123 125 124]
  [121 123 122]]

 [[ 44  50  50]
  [ 46  52  52]
  [ 48  54  54]
  ...
  [108 110 109]
  [120 122 121]
  [113 115 114]]

 ...

 [[ 55  59  58]
  [ 60  64  63]
  [ 60  64  63]
  ...
  [105 127 148]
  [104 126 147]
  [103 125 148]]

 [[ 56  60  59]
  [ 58  62  61]
  [ 57  61  60]
  ...
  [101 123 144]
  [ 99 124 146]
  [ 98 123 145]]

 [[ 55  59  58]
  [ 54  58  57]
  [ 52  56  55]
  ...
  [ 97 119 142]
  [ 94 119 141]
  [ 93 118 140]]]
torch.Size([1, 3, 224, 224])


In [ ]:
np.argmax(ort_outs)

3